In [6]:
#importing tensor flow and making a construction graph
import tensorflow as tf

# reset_graph()

x = tf.Variable(5)
y = tf.Variable(4)
f = x*x*y + y + 2

#execution phase where we run a session differebt ways
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

#Using the with block to run our session
init = tf.global_variables_initializer()

with tf.Session() as sess:
#     x.initializer.run()
#     y.initializer.run()
    init.run()
    res = f.eval()
    print(res)

#using interactive session. here we dont have to set it defualt as it is set like that by design
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

In [ ]:
#in case of different graphs within the construction phase 

#making variable and checking if its added to the main graph
x1 = tf.Variable(10)
x1.graph is tf.get_default_graph()

#addding node to another graph(non default)
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is tf.get_default_graph()

In [16]:
w = tf.constant(3)
x = w + 2
p = w + 4
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15
    
    
    
#values in the array are calculated just once
with tf.Session() as sess:
    y_val, z_val, p_val = sess.run([y, z, p])
    print(y_val)  # 10
    print(z_val)  # 15
    print(p_val)




10
15
10
15
7


In [23]:
import numpy as np
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

#with tensorflow and normal equation
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print('using tensor, theta is ', theta_value)
    
#Doing with numpy only    
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)


print('numpy theta ', theta_numpy)

#doing with scikitlearn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print('scikit learn', np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

using tensor, theta is  [[-3.6894890e+01]
 [ 4.3661433e-01]
 [ 9.4453208e-03]
 [-1.0704148e-01]
 [ 6.4345831e-01]
 [-3.9632569e-06]
 [-3.7880042e-03]
 [-4.2093179e-01]
 [-4.3400639e-01]]
numpy theta  [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]
scikit learn [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [30]:
#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# print(scaled_housing_data_plus_bias.mean(axis=0))
# print(scaled_housing_data_plus_bias.mean(axis=1))
# print(scaled_housing_data_plus_bias.mean())
# print(scaled_housing_data_plus_bias.shape)



tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    my = training_op.eval()
    
    
print(my)
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705
[[ 2.06855226e+00]
 [ 7.74153352e-01]
 [ 1.31169230e-01]
 [-1.18057005e-01]
 [ 1.64984047e-01]
 [ 7.34676723e-04]
 [-3.91941592e-02]
 [-8.61454606e-01]
 [-8.23589861e-01]]
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44081801e-04]
 [-3.91945131e-02]
 [-8.61356556e-01]
 [-8.23479712e-01]]


In [8]:
#using auto diff 

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m,n = housing.data.shape


#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#trying builtin gradeints function
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

#trying the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)



init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [50]:
#using placeholder for mini batch descents

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape


#doing linear regression by using gradient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


X = tf.placeholder(tf.float32, shape = (None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="thetas")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()

def fetch_data(batch_num,  batch_size):
    start = batch_num * batch_size
    X_batch = scaled_housing_data_plus_bias[start:start + batch_size]
    y_batch = housing.target.reshape(-1, 1)[start:start + batch_size] 
    return X_batch, y_batch

#-----------random mini batches (book)------------
# def fetch_batch(epoch, batch_index, batch_size):
#     np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
#     indices = np.random.randint(m, size=batch_size)  # not shown
#     X_batch = scaled_housing_data_plus_bias[indices] # not shown
#     y_batch = housing.target.reshape(-1, 1)[indices] # not shown
#     return X_batch, y_batch
    
    
with tf.Session() as sess:
    
    sess.run(init)
    
    for epoch in range(n_epochs):
        for a in range(n_batches ):
            X_batch, y_batch = fetch_data(a, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  


        if epoch % 100 == 0:
            print('---------------------------')
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            
    best_theta = theta.eval()
    
print(best_theta)
        

---------------------------
Epoch 0 MSE = 0.14887354
---------------------------
Epoch 100 MSE = 0.124388054
---------------------------
Epoch 200 MSE = 0.124388054
---------------------------
Epoch 300 MSE = 0.124388054
---------------------------
Epoch 400 MSE = 0.124388054
---------------------------
Epoch 500 MSE = 0.124388054
---------------------------
Epoch 600 MSE = 0.124388054
---------------------------
Epoch 700 MSE = 0.124388054
---------------------------
Epoch 800 MSE = 0.124388054
---------------------------
Epoch 900 MSE = 0.124388054
[[ 2.002909  ]
 [ 0.78347117]
 [ 0.13612016]
 [-0.2301369 ]
 [ 0.27156723]
 [-0.00706378]
 [-0.01321159]
 [-0.8690103 ]
 [-0.8959996 ]]


In [48]:
np.random.seed(206)  # not shown in the book
indices = np.random.randint(m, size=batch_size)  # not shown
#     X_batch = scaled_housing_data_plus_bias[indices] # not shown
#     y_batch = housing.target.reshape(-1, 1)[indices] # not shown


In [32]:
indices[99:109]

array([17535])